<div style="display:flex;">
<img alt="New Relic" style="display:block;height:64px" src="https://gitlab.com/softbutterfly/open-source/open-source-office/-/raw/master/banners/borderless/brands/new_relic.png" />
<img alt="SoftButterfly" style="display:block;height:64px;margin-left:auto" src="https://gitlab.com/softbutterfly/open-source/open-source-office/-/raw/master/banners/borderless/softbutterfly.png" />
</div>

# New Relic Playground 5: Drop Rules

## Imports

Python Imports

In [ ]:
import os
from datetime import datetime

Third-party libraries

In [ ]:
import dotenv
import pandas as pd
from newrelic_sb_sdk.client import NewRelicGqlClient
from newrelic_sb_sdk.graphql import nerdgraph
from newrelic_sb_sdk.graphql.objects import RootMutationType, RootQueryType
from newrelic_sb_sdk.utils.response import get_response_data, print_response
from sgqlc.operation import Operation

## Client setup

To setup the client, first we need to open load the credentials from environment variables.

In [ ]:
env_file = "../.env"

dotenv.load_dotenv(env_file)

NEW_RELIC_USER_KEY = os.environ.get("NEW_RELIC_USER_KEY", None)

if NEW_RELIC_USER_KEY is None:
    raise ValueError("Environment variable NEW_RELIC_USER_KEY is not set.")

NEW_RELIC_ACCOUNT_ID = os.environ.get("NEW_RELIC_ACCOUNT_ID", None)

if NEW_RELIC_ACCOUNT_ID is None:
    raise ValueError("Environment variable NEW_RELIC_ACCOUNT_ID is not set.")

With environment varaibles loaded, we can proceed to instantiate the client

In [ ]:
newrelic = NewRelicGqlClient(new_relic_user_key=NEW_RELIC_USER_KEY)

## Client testing

In order to use an test the client we need configure `query_type` and `mutation_type` for the `nerdgraph` schema

In [ ]:
nerdgraph.query_type = RootQueryType
nerdgraph.mutation_type = RootMutationType

For testing we will use a simple query in GraphQL to get the atttributes from our user

```gql
query {
  actor {
    user {
      email
      id
      name
    }
  }
}
```

This query will be build from the `nerdgraph` schema.

In [ ]:
# Declare an operation from the `query_type` in `nerdgraph`
operation = Operation(nerdgraph.query_type)

# Get the fields `id`, `email`, `name` from the `user` entity inside `actor`
operation.actor.user.__fields__("id", "email", "name")

This operation can be transformed into a GraphQL query

In [ ]:
query = operation.__to_graphql__()
print(query)

And this query is the one we send to be executed by our client.

In [ ]:
response = newrelic.execute(query)

The response data obtained is

In [ ]:
print_response(response)

We can also use the raw query directly written by hand. For this is recomendable to use the `build_query` method in order to get a clean query.

In [ ]:
query = newrelic.build_query(
    """
        {
            actor {
                user {
                    email
                    name
                    id
                }
            }
        }
    """
)

And execute directly, as in the previous execution, with the same result

In [ ]:
response = newrelic.execute(query)
print_response(response)

## Playground area

### Get data from NRQL

In [ ]:
template = """
{
    actor {
        account(id: %(account_id)d) {
            name
            id
        }
        nrql(
            accounts: %(account_id)d,
            query: "FROM Metric SELECT * WHERE integrationName = 'nri-prometheus'"
        ) {
            results
        }
    }
}

"""

params = {
    "account_id": int(NEW_RELIC_ACCOUNT_ID),
}

query = newrelic.build_query(
    template,
    params=params,
)

In [ ]:
response = newrelic.execute(query)

In [ ]:
account_details = get_response_data(
    response,
    key_path="account",
    action="actor",
)
account_details

In [ ]:
nrql_results = get_response_data(
    response,
    key_path="nrql:results",
    action="actor",
)
len(nrql_results)

In [ ]:
nrql_results[0]

In [ ]:
nrql_results_df = pd.DataFrame(
    data=nrql_results,
)
nrql_results_df

### Creating drop rule

In [ ]:
template = """
mutation {
    nrqlDropRulesCreate(
        accountId: %(account_id)d,
        rules: {
            action: DROP_DATA,
            description: "Drop dat from Prometheus integration.",
            nrql: "FROM Metric SELECT * WHERE integrationName = 'nri-prometheus'"
        }
    ) {
        failures {
            error {
                description
                reason
            }
            submitted {
                accountId
                action
                description
                nrql
            }
        }
        successes {
            id
            source
            nrql
        }
    }
}

"""

params = {
    "account_id": int(NEW_RELIC_ACCOUNT_ID),
}

query = newrelic.build_query(
    template,
    params=params,
)

In [ ]:
response = newrelic.execute(query)

In [ ]:
failures = get_response_data(
    response,
    key_path="failures",
    action="nrqlDropRulesCreate",
)
failures

In [ ]:
successes = get_response_data(
    response,
    key_path="successes",
    action="nrqlDropRulesCreate",
)
successes

### Look at ingested data

In [ ]:
template = """
{
    actor {
        account(id: %(account_id)d) {
            name
            id
        }
        nrql(
            accounts: %(account_id)d,
            query: "FROM Metric SELECT latest(timestamp) WHERE integrationName = 'nri-prometheus'"
        ) {
            results
        }
    }
}
"""

params = {
    "account_id": int(NEW_RELIC_ACCOUNT_ID),
}

query = newrelic.build_query(
    template,
    params=params,
)

In [ ]:
response = newrelic.execute(query)

In [ ]:
latest_timestamp = get_response_data(
    response,
    key_path="nrql:results:0:latest.timestamp",
    action="actor",
)
latest_timestamp

In [ ]:
latest_ingest_date = datetime.fromtimestamp(latest_timestamp / 1000)


print(latest_ingest_date)